In [1]:
import pandas as pd
from pandasql import sqldf

In [2]:
#Setting up the sqldf from pandasql for easier access
def sql(q):
    return sqldf(q, globals())

In [3]:
#This data is a sample of 10000 observations, from bigger data sets, due to the amount of memory needed to load all the data.
# Refer to the get data bash script
vehicles = pd.read_csv('../data/vehicle.csv')
people = pd.read_csv('../data/person.csv')
crashes = pd.read_csv('../data/crash.csv', parse_dates=['CRASH DATE', 'CRASH TIME'], infer_datetime_format=True)

COLLISION_ID is the primary key in the crashes tables and it serves as foreign key in the people and vehicles tables.

In [4]:
#Which hours of the day seems to have the most accidents(top 5)?
sql('''
    SELECT
        strftime("%H",`CRASH TIME`) AS Hour,
        COUNT(*) AS Total
    FROM
        crashes
    GROUP BY Hour
    ORDER BY Total DESC
    LIMIT 5;
''')

,Hour,Total
0,16,765
1,17,702
2,18,694
3,14,679
4,15,641


Most of the collisions occurred on peak hours, usually when it's comming back from work.

In [5]:
# What is the total amount people injured and the average age by person type when there is at least one or more people other than the driver?
sql('''
    SELECT
        PERSON_TYPE,
        ROUND(AVG(PERSON_AGE),0) AS Mean_Age,
        SUM(is_injured) AS Total_Injured        
    FROM
        (SELECT
            c.`COLLISION_ID`,            
            c.`NUMBER OF PERSONS INJURED`,
            p.PERSON_TYPE,
            p.PERSON_AGE,
            CASE WHEN p.PERSON_INJURY = "Injured" THEN 1
                ELSE 0
            END AS is_injured,
            v.VEHICLE_OCCUPANTS              
        FROM crashes c 
            INNER JOIN people p 
            ON c.COLLISION_ID = p.COLLISION_ID
            INNER JOIN vehicles v
            ON c.COLLISION_ID = v.COLLISION_ID
        WHERE c.`ON STREET NAME` IS NOT NULL) AS collisions
    WHERE VEHICLE_OCCUPANTS >= 2
    GROUP BY PERSON_TYPE
    ORDER BY Total_Injured DESC;
''')

,PERSON_TYPE,Mean_Age,Total_Injured
0,Occupant,36.0,750
1,Bicyclist,31.0,36
2,Pedestrian,39.0,17


It seems that occupants are the most common injured people on a crash and the mean age accross all categories seems to be between 30 and 40 years old.

In [6]:
#How many people were involved in the collisions by each known vehicle type from january 2020 to may 2020, by the top 10 types?
sql('''
    SELECT 
        v.VEHICLE_TYPE,
        COUNT(p.UNIQUE_ID) AS Total_People        
    FROM crashes c
        INNER JOIN people p
        ON c.COLLISION_ID = p.COLLISION_ID
        INNER JOIN vehicles v
        ON c.COLLISION_ID = v.COLLISION_ID
    WHERE v.VEHICLE_TYPE IS NOT NULL
        AND (STRFTIME("%Y-%m-%d",c.`CRASH DATE`) BETWEEN "2020-01-01" AND "2020-05-31")
    GROUP BY v.VEHICLE_TYPE
    ORDER BY Total_People DESC
    LIMIT 10;        
''')

,VEHICLE_TYPE,Total_People
0,Sedan,607
1,Station Wagon/Sport Utility Vehicle,398
2,Bike,34
3,Pick-up Truck,23
4,Motorcycle,22
5,Taxi,14
6,Tractor Truck Diesel,8
7,E-Scooter,6
8,Bus,6
9,Box Truck,6


Most incident with the most people involved happen with sedans and station wagons on the first 5 months of 2020.